---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():

        #G1 = P1_Graphs[4]
        #degrees = G1.degree()
        #degree_values = sorted(set(degrees.values()))
        #histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G1)) for i in degree_values]
    
        #plt.plot(degree_values, histogram, 'o')
        #plt.xscale('log')
        #plt.yscale('log')
        
        answer = ['PA', 'SW_L', 'SW_L','PA', 'SW_H']
        
        return answer
  
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [90]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [95]:
from sklearn.neural_network import *
from sklearn.preprocessing import *

def salary_predictions():
    
    df = pd.DataFrame(index = G.nodes())
    
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['Degree'] = pd.Series(G.degree())
    df['Degree centrality'] = pd.Series(nx.degree_centrality(G))
    df['Closeness centrality'] = pd.Series(nx.closeness_centrality(G, normalized = True))
    df['Betweenness centrality'] = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))
    df['PageRank'] = pd.Series(nx.pagerank(G, alpha = 0.85))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['Management Salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    
    X = df.dropna(subset=['Management Salary'])
    X_train = X.iloc[:, :-1]
    y_train = X.iloc[:, -1]
    y_train = y_train.astype('int')
    Y = df[df['Management Salary'].isnull()]
    X_test = Y.iloc[:, :-1]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 10],
                        alpha = 0.001,
                        random_state = 0,
                        solver = 'lbfgs',
                        verbose = 0)
    clf.fit(X_train_scaled, y_train)
    y_proba = clf.predict_proba(X_test_scaled)[:,1]
    
    X_test['proba_scores'] = y_proba
    
    return X_test.iloc[:, -1]

salary_predictions()

1       8.500897e-03
2       8.811451e-01
5       1.000000e+00
8       1.261637e-01
14      5.753242e-02
18      1.125002e-01
27      4.059840e-01
30      4.890655e-01
31      1.131439e-01
34      1.623634e-01
37      8.617024e-03
40      3.389924e-02
45      1.416472e-01
54      8.388874e-01
55      9.761858e-01
60      6.899933e-02
62      9.999938e-01
65      9.979222e-01
77      6.720257e-03
79      1.961683e-02
97      1.643166e-02
101     3.430189e-03
103     3.146222e-01
108     3.274545e-02
113     1.212267e-01
122     8.663571e-04
141     4.191352e-01
142     6.480601e-01
144     5.601172e-04
145     9.250038e-01
            ...     
913     2.013110e-04
914     2.913169e-03
915     1.083190e-04
918     1.598062e-01
923     7.157355e-06
926     7.826778e-03
931     2.637029e-04
934     4.250406e-07
939     2.688766e-06
944     5.262571e-05
945     2.350030e-05
947     1.585689e-01
950     8.827125e-03
951     1.954772e-05
953     8.167314e-05
959     1.186877e-03
962     2.966

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [188]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [197]:
from sklearn.neural_network import *
from sklearn.preprocessing import *

def new_connections_predictions():

    common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G)]
    df1 = pd.DataFrame(index = [(item[0],item[1]) for item in common_neigh])
    df1['common_neigh'] = [item[2] for item in common_neigh]
    
    Jaccard_coef = list(nx.jaccard_coefficient(G))
    df2 = pd.DataFrame(index = [(item[0],item[1]) for item in Jaccard_coef])
    df2['Jaccard_coef'] = [item[2] for item in Jaccard_coef]
    
    Resource_allocation = list(nx.resource_allocation_index(G))
    df3 = pd.DataFrame(index = [(item[0],item[1]) for item in Resource_allocation])
    df3['Resource_allocation'] = [item[2] for item in Resource_allocation]
    
    preferential_attachment = list(nx.preferential_attachment(G))
    df4 = pd.DataFrame(index = [(item[0],item[1]) for item in preferential_attachment])
    df4['preferential_attachment'] = [item[2] for item in preferential_attachment]
    
    connections = df1.join(df2, how = 'inner').join(df3, how = 'inner').join(df4, how = 'inner')
    future_connections_mixed = future_connections.join(connections, how = 'left')
    
    future_connections_missing = future_connections_mixed[future_connections_mixed['Future Connection'].isnull()]
    future_connections_okay = future_connections_mixed[~future_connections['Future Connection'].isnull()] 
    
    X_train = future_connections_okay.drop(['Future Connection'], axis = 1)
    
    remove_list = ['common_neigh', 'Jaccard_coef', 'Resource_allocation', 'preferential_attachment']
    Y_train = future_connections_okay.drop(remove_list, axis = 1)
    
    X_test = future_connections_missing.drop(['Future Connection'], axis = 1)
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 10],
                          alpha = 0.001,
                          random_state = 0,
                          solver = 'lbfgs',
                          verbose = 0)
    
    clf.fit(X_train_scaled, Y_train)
    
    y_proba_test = clf.predict_proba(X_test_scaled)[:,1]
    X_test['proba_scores'] = y_proba_test
    
    return X_test.iloc[:, -1]

new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(107, 348)    0.028110
(542, 751)    0.011176
(20, 426)     0.566451
(50, 989)     0.011259
(942, 986)    0.011349
(324, 857)    0.011248
(13, 710)     0.187435
(19, 271)     0.120935
(319, 878)    0.011288
(659, 707)    0.011185
(49, 843)     0.011314
(208, 893)    0.011205
(377, 469)    0.008105
(405, 999)    0.019777
(129, 740)    0.016577
(292, 618)    0.030049
(239, 689)    0.011294
(359, 373)    0.009302
(53, 523)     0.038734
(276, 984)    0.011310
(202, 997)    0.011327
(604, 619)    0.164244
(270, 911)    0.011292
(261, 481)    0.073495
(200, 450)    0.903473
(213, 634)    0.011151
(644, 735)    0.101979
(346, 553)    0.010945
(521, 738)    0.010553
(422, 953)    0.020272
                ...   
(672, 848)    0.011292
(28, 127)     0.938907
(202, 661)    0.010910
(54, 195)     0.999991
(295, 864)    0.011235
(814, 936)    0.011164
(839, 874)    0.011349
(139, 843)    0.011242
(461, 544)    0.010282
(68, 487)     0.010234
(622, 932)    0.011215
(504, 936)    0.017456
(479, 528) 